In [171]:
import requests
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
import pymorphy2
from collections import defaultdict
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np

In [183]:
morph = pymorphy2.MorphAnalyzer()
def normalize_split_sentence(sentence):
    sentence_dict = defaultdict(int)
    for word in word_tokenize(re.sub(r'[^\w]|[0-9]+', ' ', sentence, flags=re.U), language='russian'):
        sentence_dict[morph.parse(word)[0].normal_form] += 1
    return sentence_dict

In [184]:
def construct_base(urls):
    if type(urls) != list:
        urls = [urls]
    sentences = []
    sentences_normal = []
    for url in urls:
        wiki = requests.get(url)
        soup = BeautifulSoup(wiki.text, 'html.parser')
        content = soup.find(id="mw-content-text")
        for p in content.findAll('p', recursive=False):
            for sentence in sent_tokenize(p.text, language='russian'):
                sentences.append(sentence)
                sentences_normal.append(normalize_split_sentence(sentence))
    dict_to_vec = DictVectorizer()
    vec_to_weights = TfidfTransformer()
    sentences_weights = vec_to_weights.fit_transform(dict_to_vec.fit_transform(sentences_normal)).toarray()
    return np.array(sentences), sentences_weights, dict_to_vec, vec_to_weights

In [185]:
def sort_sentences_by_query(urls, query):
    sentences, sentences_weights, dict_to_vec, vec_to_weights = construct_base(urls)
    query_weights = vec_to_weights.transform(dict_to_vec.transform(normalize_split_sentence(query))).toarray()
    cosine = np.sum(sentences_weights*query_weights, axis=1)
    return sentences[cosine.argsort()[::-1]]

In [188]:
def print_sorted(sentences):
    for i, sent in enumerate(sentences):
        print i+1, ':\t', sent

<center><font size="5">Факт <b>"Минирование берегов спасает популяцию пингвинов"</b> содержится в 1-м предложении.</font></center>

In [195]:
url = u'https://ru.wikipedia.org/wiki/FMK-3_(мина)'
query = u'Минирование берегов спасает популяцию пингвинов.'
sentences_sorted = sort_sentences_by_query(url, query)
print_sorted(sentences_sorted[:5])

1 :	Эти огороженные колючей проволокой территории неожиданно способствовали увеличению популяции пингвинов, находившихся здесь на грани исчезновения[10].
2 :	Вес пингвина оказался недостачным для срабатывания мин.
3 :	Места обитания оказались настолько популярными и прибыльными для экотуризма, что появились люди, протестующие против обезвреживания мин[11].
4 :	Таким образом, они могут благополучно размножаться без помех со стороны людей.
5 :	На 2000 год на Фолклендских островах на 117 минных полях общей площадью 20 км2 находилось 25 тысяч необезвреженных мин[8], в том числе противотанковых FMK-3[9].


<center><font size="5">Факт <b>"Для профессионального японского футболиста очень важна причёска"</b> содержится во 2-м предложении.</font></center>

In [194]:
url = u'https://ru.wikipedia.org/wiki/Футбол_в_Японии'
query = u'Для профессионального японского футболиста очень важна причёска.'
sentences_sorted = sort_sentences_by_query(url, query)
print_sorted(sentences_sorted[:5])

1 :	В Японской профессиональной футбольной лиге зарегистрировано более 900 000 футболистов, а количество профессиональных футбольных клубов в стране превышает 40[2].
2 :	Для привлечения фанатов используются разнообразные маркетинговые стратегии: изначально при выборе клубов-основателей перед созданием Джей-лиги выбор был сделан в пользу провинциальных команд, что увеличило популярность футбола в сельской местности; кроме того, особое внимание уделяется образу игроков для завлечения фанаток, в частности в модных журналах регулярно выходят обзоры причёсок и одежды футболистов[8].
3 :	Зарплаты футболистов составляют около половины затрат[7].
4 :	Японская профессиональная футбольная лига является одной из сильнейших в Азии, она дважды (в 2007 и 2008) выигрывала кубок АФК.
5 :	На вершине структуры футбольных лиг находятся первый и второй дивизионы Японской профессиональной футбольной лиги (называемой также Джей-лигой, от англ. J. League), ниже которых располагаются полупрофессиональная Япон

<center><font size="5">Факт <b>"Мексиканская империя терпеливо ждала, пока страны Центральной Америки согласятся быть аннексированными"</b> содержится во 2-м предложении.</font></center>

In [193]:
url = u'https://ru.wikipedia.org/wiki/Первая_Мексиканская_империя'
url2 = u'https://ru.wikipedia.org/wiki/Центральная_Америка'
url3 = u'https://ru.wikipedia.org/wiki/Аннексия_Центральной_Америки_Мексикой'

query = u'Мексиканская империя терпеливо ждала, пока страны Центральной Америки согласятся быть аннексированными.'

sentences_sorted = sort_sentences_by_query(url3, query)
print_sorted(sentences_sorted[:5])

1 :	1 июля 1823 года была принята Декларация о полной независимости Центральной Америки, вновь отделившая её от Мексики.
2 :	3 декабря 1821 года, Гаинса написал ответное письмо к Итурбиде, в котором указал, что для принятия решения ему необходимо выяснить мнение представителей всех административных единиц Центральной Америки, и попросил срок до 3 января 1822 года, а до этого времени мексиканские войска, уже подошедшие к границе[2], не должны были вторгаться в Центральную Америку.
3 :	Центральная Америка была административно выделена в качестве генерал-капитанства Гватемалы внутри вицекоролевства Новой Испании (Мексики) в 1609 году.
4 :	Однако 29 октября 1821 года Итурбиде — уже в качестве регента Мексиканской империи — отправил письмо генерал-капитану Временной консультативной хунты Центральной Америки Габино Гаинсе (исп.) с предложением о присоединении к Мексике[1] на основе «трёх гарантий», указанных в Кордовском договоре.
5 :	Однако уже 19 марта 1823 года Итурбиде отрёкся от престол